In [1]:
import numpy as np
import logging 
import sys
import os
import h5py
import math
from math import radians, cos, sin
from scipy.cluster.vq import whiten, kmeans, kmeans2

logger = logging.getLogger(__name__)
disp=False

In [2]:
### read the data we have collected
with h5py.File('../scripts/{}.h5'.format('joints_data'), 'r+') as f:
    pos_grp = f['workspace_coords']
    q       = pos_grp['joint_positions'].value
    qdot    = pos_grp['joint_velocities'].value
    targ_grp = f['workspace_targets']
    qstar   = targ_grp['joint_positions'].value
    qdot_star= targ_grp['joint_velocities'].value

In [3]:
q1 = q[0:7,:]
q2 = q[7:14,:]
q1dot = qdot[0:7,:]
q2dot = qdot[7:14,:]
pts = dict(pts1=dict(q=q1, qdot=q1dot),pts2=dict(q=q2, qdot=q2dot),targets=dict(q=qstar, qdot=qdot_star),)

In [4]:
np.set_printoptions(suppress=True)

if disp:
    print(pts)

### convert from joint space to cartesian coordinates in the end effector/tool frame

In [5]:
#### defining D-H constants
"""lc, lf are in millimeters"""
lc, lf = 300, 350

pts_cartesian = {
    'pts1': {
        'x': None, 'y': None, 'z': None,
        'xdot': None, 'ydot': None, 'zdot': None,
    },
    'pts2': {
        'x': None, 'y': None, 'z': None,
        'xdot': None, 'ydot': None, 'zdot': None,
    },
    'targets': {
        'x': None, 'y': None, 'z': None,
        'xdot': None, 'ydot': None, 'zdot': None,
    }
}

In [6]:
pts['pts1'].keys()

dict_keys(['q', 'qdot'])

In [7]:
way_pts = [k for k, v in pts.items()]
for way_pt in way_pts:
    # get the positional points
    px_l = [cos(x) for x in pts[way_pt]['q'][:,0]]
    px_r1 = [lc*cos(x) for x in  pts[way_pt]['q'][:,1]]
    px_r2 = [lf*cos(x+y) for x,y in  zip(pts[way_pt]['q'][:,1], pts[way_pt]['q'][:,2])]
    px_r = np.array(px_r1) - px_r2
    pts_cartesian[way_pt]['x'] = np.array(px_l) * np.array(px_r)

    py_l = [sin(x) for x in pts[way_pt]['q'][:,0]]
    pts_cartesian[way_pt]['y'] = np.array(py_l) * np.array(px_r)

    pz_l = [-lc * sin(x) for x in pts[way_pt]['q'][:,1]]
    pz_r = [lf * cos(x+y) for x, y in zip(pts[way_pt]['q'][:,1], pts[way_pt]['q'][:,2])]
    pts_cartesian[way_pt]['z'] = np.array(pz_l) - np.array(pz_r)
    
    # get the velocities
    px_l = [cos(x) for x in pts[way_pt]['qdot'][:,0]]
    px_r1 = [lc*cos(x) for x in  pts[way_pt]['qdot'][:,1]]
    px_r2 = [lf*cos(x+y) for x,y in  zip(pts[way_pt]['qdot'][:,1], pts[way_pt]['qdot'][:,2])]
    px_r = np.array(px_r1) - px_r2
    pts_cartesian[way_pt]['xdot'] = np.array(px_l) * np.array(px_r)

    py_l = [sin(x) for x in pts[way_pt]['qdot'][:,0]]
    pts_cartesian[way_pt]['ydot'] = np.array(py_l) * np.array(px_r)

    pz_l = [-lc * sin(x) for x in pts[way_pt]['qdot'][:,1]]
    pz_r = [lf * cos(x+y) for x, y in zip(pts[way_pt]['qdot'][:,1], pts[way_pt]['qdot'][:,2])]
    pts_cartesian[way_pt]['zdot'] = np.array(pz_l) - np.array(pz_r)
    
# print(pts_cartesian)

#### gather all the cartesian points and velocities
+ data is thus arranged:
  - data = [
             \zeta^0, \zeta^1, \zeta^\star; 
             \dot{\zeta^0}, \dot{\zeta^1}, \dot{\zeta^\star}
            ]
  - where \zeta \in R^n, n being the dimension in cartesian coordinates of the 
  - note that our data is in 2D for now, i.e. x and y axes in Cartesian space

In [8]:
x = np.hstack([pts_cartesian['pts1']['x'], pts_cartesian['pts2']['x'], pts_cartesian['targets']['x']])
y = np.hstack([pts_cartesian['pts1']['y'], pts_cartesian['pts2']['y'], pts_cartesian['targets']['y']])
z = np.hstack([pts_cartesian['pts1']['z'], pts_cartesian['pts2']['z'], pts_cartesian['targets']['z']])

# xd = np.hstack([pts_cartesian['pts1']['xdot'], pts_cartesian['pts2']['xdot'], pts_cartesian['targets']['xdot']])
# yd = np.hstack([pts_cartesian['pts1']['ydot'], pts_cartesian['pts2']['ydot'], pts_cartesian['targets']['ydot']])
# zd = np.hstack([pts_cartesian['pts1']['zdot'], pts_cartesian['pts2']['zdot'], pts_cartesian['targets']['zdot']])
xd,yd,zd = [[] for _ in range(3)]
for i in range(len(x)-1):
    xd.append((x[i+1]-x[i])/0.1)
    yd.append((y[i+1]-y[i])/0.1)
    zd.append((z[i+1]-z[i])/0.1)
xd.append(0)
yd.append(0)
zd.append(0)

In [9]:
# x.shape, y.shape, z.shape
data = np.vstack([x,y,np.array(xd),np.array(yd)])
#data = np.vstack([x,y,z])
# print(data.shape)
filename = '../scripts/{}.h5'.format('torobo_processed_data')
os.remove(filename) if os.path.isfile(filename) else None
# time.sleep(4)
with h5py.File(filename, 'w') as f:
    pos_grp = f.create_group('data')
    pos_grp.create_dataset("data", data=data, dtype=np.float32, compression="gzip", compression_opts=9)

#### now compute the priors, mus and sigmas of the Gaussian Mixture Model

In [10]:
def kmeans_init_data(data, num_states=2):
    """
        num_states: number of k centroids
    """
    whitened = whiten(data)
    centroids, labels = kmeans2(whitened, num_states, iter=500, thresh=1e-05, check_finite=True)
    
    return centroids, labels

In [136]:
""" This file defines a Gaussian mixture model class. """
import logging

import numpy as np
import scipy.linalg


LOGGER = logging.getLogger(__name__)

def logsum(vec, axis=0, keepdims=True):
    #TODO: Add a docstring.
    maxv = np.max(vec, axis=axis, keepdims=keepdims)
    maxv[maxv == -float('inf')] = 0
    return np.log(np.sum(np.exp(vec-maxv), axis=axis, keepdims=keepdims)) + maxv

def check_sigma(A):
    """
        checks if the sigma matrix is symmetric
        positive definite before inverting via cholesky decomposition
    """
    eigval = np.linalg.eigh(A)[0]
    if np.array_equal(A, A.T) and np.all(eigval>0):
        # LOGGER.debug("sigma is pos. def. Computing cholesky factorization")
        return A
    else:
        # find lowest eigen value
        eta = 1e-6  # regularizer for matrix multiplier
        low = np.amin(np.sort(eigval))
        Anew = low * A + eta * np.eye(A.shape[0])
        return Anew

class GMM(object):
    """ Gaussian Mixture Model. """
    def __init__(self, init_sequential=False, eigreg=False, warmstart=True):
        self.init_sequential = init_sequential
        self.eigreg = eigreg
        self.warmstart = warmstart
        self.sigma = None

    def inference(self, pts):
        """
        Evaluate dynamics prior.
        Args:
            pts: A N x D array of points.
        """
        # Compute posterior cluster weights.
        logwts = self.clusterwts(pts)

        # Compute posterior mean and covariance.
        mu0, Phi = self.moments(logwts)

        # Set hyperparameters.
        m = self.N
        n0 = m - 2 - mu0.shape[0]

        # Normalize.
        m = float(m) / self.N
        n0 = float(n0) / self.N
        return mu0, Phi, m, n0

    def clusterwts(self, data):
        """
        Compute cluster weights for specified points under GMM.
        Args:
            data: An N x D array of points
        Returns:
            A K x 1 array of average cluster log probabilities.
        """
        # Compute probability of each point under each cluster.
        logobs = self.estep(data)

        # Renormalize to get cluster weights.
        logwts = logobs - logsum(logobs, axis=1)

        # Average the cluster probabilities.
        logwts = logsum(logwts, axis=0) - np.log(data.shape[0])
        return logwts.T

    def estep(self, data):
        """
        Compute log observation probabilities under GMM.
        Args:
            data: A N x D array of points.
        Returns:
            logobs: A N x K array of log probabilities (for each point
                on each cluster).
        """
        # Constants.
        N, D = data.shape
        K = self.sigma.shape[0]

        logobs = -0.5*np.ones((N, K))*D*np.log(2*np.pi)
        for i in range(K):
            mu, sigma = self.mu[i], self.sigma[i]
            sigma = sigma
            L = scipy.linalg.cholesky(sigma, lower=True)
            logobs[:, i] -= np.sum(np.log(np.diag(L)))
            diff = (data - mu).T
            soln = scipy.linalg.solve_triangular(L, diff, lower=True)
            logobs[:, i] -= 0.5*np.sum(soln**2, axis=0)

        logobs += self.logmass.T
        return logobs

    def moments(self, logwts):
        """
            Compute the moments of the cluster mixture with logwts.
            Args:
                logwts: A K x 1 array of log cluster probabilities.
            Returns:
                mu: A (D,) mean vector.
                sigma: A D x D covariance matrix.
        """
        # Exponentiate.
        wts = np.exp(logwts)

        # Compute overall mean.
        mu = np.sum(self.mu * wts, axis=0)

        # Compute overall covariance.
        diff = self.mu - np.expand_dims(mu, axis=0)
        diff_expand = np.expand_dims(self.mu, axis=1) * \
                np.expand_dims(diff, axis=2)
        wts_expand = np.expand_dims(wts, axis=2)
        sigma = np.sum((self.sigma + diff_expand) * wts_expand, axis=0)
        return mu, sigma

    def update(self, data, K, max_iterations=100):
        """
        Run EM to update clusters.
        Args:
            data: An N x D data matrix, where N = number of data points.
            K: Number of clusters to use.
        """
        # Constants.
        N  = data.shape[0]
        Do = data.shape[1]

        LOGGER.debug('Fitting GMM with %d clusters on %d points.', K, N)

        if (not self.warmstart or self.sigma is None or K != self.sigma.shape[0]):
            # Initialization.
            LOGGER.debug('Initializing GMM.')
            self.sigma = np.zeros((K, Do, Do))
            self.mu = np.zeros((K, Do))
            self.logmass = np.log(1.0 / K) * np.ones((K, 1))
            self.mass = (1.0 / K) * np.ones((K, 1))
            self.N = data.shape[0]
            N = self.N

            # Set initial cluster indices.
            if not self.init_sequential:
                cidx = np.random.randint(0, K, size=(1, N))
            else:
                raise NotImplementedError()

            # Initialize.
            for i in range(K):
                cluster_idx = (cidx == i)[0]
                mu = np.mean(data[cluster_idx, :], axis=0)
                diff = (data[cluster_idx, :] - mu).T
                sigma = (1.0 / K) * (diff.dot(diff.T))
                self.mu[i, :] = mu
                self.sigma[i, :, :] = sigma + np.eye(Do) * 2e-6

        prevll = -float('inf')
        for itr in range(max_iterations):
            # E-step: compute cluster probabilities.
            logobs = self.estep(data)

            # Compute log-likelihood.
            ll = np.sum(logsum(logobs, axis=1))
            LOGGER.debug('GMM itr %d/%d. Log likelihood: %f',
                         itr, max_iterations, ll)
            if ll < prevll:
                # TODO: Why does log-likelihood decrease sometimes?
                LOGGER.debug('Log-likelihood decreased! Ending on itr=%d/%d',
                             itr, max_iterations)
                break
            if np.abs(ll-prevll) < 1e-5*prevll:
                LOGGER.debug('GMM converged on itr=%d/%d',
                             itr, max_iterations)
                break
            prevll = ll

            # Renormalize to get cluster weights.
            logw = logobs - logsum(logobs, axis=1)
            assert logw.shape == (N, K)

            # Renormalize again to get weights for refitting clusters.
            logwn = logw - logsum(logw, axis=0)
            assert logwn.shape == (N, K)
            w = np.exp(logwn)

            # M-step: update clusters.
            # Fit cluster mass.
            self.logmass = logsum(logw, axis=0).T
            self.logmass = self.logmass - logsum(self.logmass, axis=0)
            assert self.logmass.shape == (K, 1)
            self.mass = np.exp(self.logmass)

            # Reboot small clusters.
            w[:, (self.mass < (1.0 / K) * 1e-4)[:, 0]] = 1.0 / N
            # Fit cluster means.
            w_expand = np.expand_dims(w, axis=2)
            data_expand = np.expand_dims(data, axis=1)
            self.mu = np.sum(w_expand * data_expand, axis=0)
            # Fit covariances.
            wdata = data_expand * np.sqrt(w_expand)
            assert wdata.shape == (N, K, Do)
            for i in range(K):
                # Compute weighted outer product.
                XX = wdata[:, i, :].T.dot(wdata[:, i, :])
                mu = self.mu[i, :]
                self.sigma[i, :, :] = XX - np.outer(mu, mu)

                if self.eigreg:  # Use eigenvalue regularization.
                    raise NotImplementedError()
                else:  # Use quick and dirty regularization.
                    sigma = self.sigma[i, :, :]
                    self.sigma[i, :, :] = 0.5 * (sigma + sigma.T) + \
                            1e-6 * np.eye(Do)
                    
gmm = GMM()
gmm.update(data.T, K=6, max_iterations=100)

In [137]:
mu, sigma, priors = gmm.mu, gmm.sigma, gmm.logmass

In [138]:
print(mu.shape, sigma.shape, priors.shape)

(6, 4) (6, 4, 4) (6, 1)


d = data.shape[0]/2
num_states = 4
num_var, num_data = data.shape
centers, labels = kmeans_init_data(data, num_states=num_states)
mu = centers.T

print('labels: ', labels)
print('centers: ', centers)
Priors = np.zeros((num_states))
Sigma  = np.zeros((num_states, num_states, num_states))

for i in range(num_states):
    idtemp = np.nonzero(labels==i)
    Priors[i] = len(idtemp[0])
    print('Priors: ', Priors[i])
    print('idtemp[0]: ', idtemp[0], 'labels[idtemp]: ', labels[idtemp])
    if labels[idtemp].size == 0:
        Sigma[:,:,i] = np.random.randn(num_states, num_states)
    else:
        Sigma[:,:,i] = np.cov(m=data[:, idtemp[0]])
    print('data[:, idtemp[0]]: ', data[:, idtemp[0]].T)
    print('Sigma[:,:,i]: \n', Sigma[:,:,i])
    print(1e-5 * np.diag(np.ones((num_var, 1))))
    # avoid numerical stability
    Sigma[:,:,i] = Sigma[:,:,i] + 1e-5 * np.diag(np.ones([num_var, 1]))

Priors = np.divide(Priors, sum(Priors))